# Star Trek 

In [1]:
!git clone https://github.com/jkingsman/Star-Trek-Script-Programmatics
!pip install pandas tqdm jsonlines

fatal: destination path 'Star-Trek-Script-Programmatics' already exists and is not an empty directory.
You should consider upgrading via the '/Users/ryan/github/startrek/venv/bin/python -m pip install --upgrade pip' command.


In [2]:
import pandas as pd
import json
import os
from tqdm import tqdm
from datetime import datetime as dt


In [26]:
def parse_episode(fnfn):
    ep_id = fnfn.replace('.json','').split('/processed/',1)[-1]
    series=ep_id.split('/')[0]
    ep=ep_id.split('/',1)[-1]
    o=[]
    with open(fnfn) as f:
        data=json.load(f)
        airdate = dt.fromtimestamp(data.pop('airdate')).strftime('%Y-%m-%d')
        for k,v in data.pop('schedule').items(): data[k]=v
        for scene_i,scened in enumerate(data.pop('scenes')):
            for diald in scened.pop('dialogue'):
                text=diald.pop('line')
                od = {
                    'airdate':airdate,
                    'series':series,
                    'ep':ep, 
                    'scene':scene_i+1,
                    # **data,
                    **scened,
                    'line':len(o)+1,
                    **diald,
                    'text':text,
                    
                }
                o.append(od)
    return o

In [27]:
# Test
pd.DataFrame(parse_episode('Star-Trek-Script-Programmatics/processed/ds9/s04e01.json'))

,airdate,series,ep,scene,location,line,character,text,modifier
0,1995-10-02,ds9,s04e01,1,unknown,1,unknown,In memory of\n Gregg Duffy Long\n Ronald W. Smith,NaN
1,1995-10-02,ds9,s04e01,2,Guest quarters,2,O'BRIEN,O'Brien to Sisko.,voiceover
2,1995-10-02,ds9,s04e01,2,Guest quarters,3,SISKO,Go ahead.,NaN
3,1995-10-02,ds9,s04e01,2,Guest quarters,4,O'BRIEN,We've swept all of level seventeen. No sign of...,voiceover
4,1995-10-02,ds9,s04e01,2,Guest quarters,5,SISKO,Move down to eighteen. We'll meet you there af...,NaN
...,...,...,...,...,...,...,...,...,...
808,1995-10-02,ds9,s04e01,70,Ops,809,ODO,Don't worry. I'm sure Dukat will take all the ...,NaN
809,1995-10-02,ds9,s04e01,70,Ops,810,BASHIR,"Poor Garak. Dukat's a hero, and Garak's still ...",NaN
810,1995-10-02,ds9,s04e01,70,Ops,811,DAX,"Captain, I'm receiving a priority one message ...",NaN
811,1995-10-02,ds9,s04e01,70,Ops,812,KIRA,Looks like the Klingons are here to stay.,NaN


In [28]:
def get_all_script_filenames(searchdir='Star-Trek-Script-Programmatics/processed/'):
    o=[]
    for path,folders,files in sorted(os.walk(searchdir)):
        for fn in sorted(files):
            if fn.endswith('.json') and fn.startswith('s'):
                fnfn=os.path.join(path, fn)
                o.append(fnfn)
    return o
                

In [29]:
def iter_all_lines(files = None):
    if files is None: files = get_all_script_filenames()
    for fn in tqdm(files, desc='Iterating script files'):
        yield from parse_episode(fn)


In [35]:
def get_all_lines():
    return pd.DataFrame(iter_all_lines()).sort_values(['airdate','ep','line']).rename_axis('_i')


In [37]:
df = get_all_lines()
df

Iterating script files: 100%|██████████| 509/509 [00:00<00:00, 2508.19it/s]


,airdate,series,ep,scene,location,line,character,text,modifier
_i,,,,,,,,,
67190,1987-09-28,tng,s01e01,1,unknown,1,PICARD,"Captain's log, stardate 41153.7. Our destinati...",voiceover
67191,1987-09-28,tng,s01e01,2,Bridge,2,PICARD,"You will agree, Data, that Starfleet's orders ...",NaN
67192,1987-09-28,tng,s01e01,2,Bridge,3,DATA,Difficult? Simply solve the mystery of Farpoin...,NaN
67193,1987-09-28,tng,s01e01,2,Bridge,4,PICARD,As simple as that.,NaN
67194,1987-09-28,tng,s01e01,2,Bridge,5,TROI,Farpoint Station. Even the name sounds mysteri...,NaN
...,...,...,...,...,...,...,...,...,...
200561,2001-05-23,voy,s07e25,72,Bridge,806,JANEWAY,"You'd better get down there, Tom.",NaN
200562,2001-05-23,voy,s07e25,72,Bridge,807,PARIS,"Yes, ma'am.",NaN
200563,2001-05-23,voy,s07e25,72,Bridge,808,JANEWAY,"Mister Chakotay, the helm.",NaN


In [38]:
df.to_csv('corpus.csv', index=True)

In [39]:
df.series.value_counts()

series
voy    69482
ds9    67190
tng    63894
Name: count, dtype: int64

In [40]:
df

,airdate,series,ep,scene,location,line,character,text,modifier
_i,,,,,,,,,
67190,1987-09-28,tng,s01e01,1,unknown,1,PICARD,"Captain's log, stardate 41153.7. Our destinati...",voiceover
67191,1987-09-28,tng,s01e01,2,Bridge,2,PICARD,"You will agree, Data, that Starfleet's orders ...",NaN
67192,1987-09-28,tng,s01e01,2,Bridge,3,DATA,Difficult? Simply solve the mystery of Farpoin...,NaN
67193,1987-09-28,tng,s01e01,2,Bridge,4,PICARD,As simple as that.,NaN
67194,1987-09-28,tng,s01e01,2,Bridge,5,TROI,Farpoint Station. Even the name sounds mysteri...,NaN
...,...,...,...,...,...,...,...,...,...
200561,2001-05-23,voy,s07e25,72,Bridge,806,JANEWAY,"You'd better get down there, Tom.",NaN
200562,2001-05-23,voy,s07e25,72,Bridge,807,PARIS,"Yes, ma'am.",NaN
200563,2001-05-23,voy,s07e25,72,Bridge,808,JANEWAY,"Mister Chakotay, the helm.",NaN
